# WIP Replicating AdpQ

From [Ghaffari et al 2024]

In [ ]:
pip install huggingface_hub transformers 

In [ ]:
if "COLAB_" not in "".join(os.environ.keys()):
    from huggingface_hub import login
    login() # To access gated repo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig

In [ ]:
import warnings
import gc
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Boiler plate code
import glob
from IPython.display import Javascript, display
import functools
from huggingface_hub import snapshot_download
import os

def notify_when_done(show_popup=True, message="✅ Task finished!"):
    """
    Decorator that notifies (via pop-up and/or sound) when a function finishes running.
    
    Args:
        show_popup (bool): Whether to show a pop-up message.
        play_sound (bool): Whether to play a notification sound.
        message (str): Message to show in the pop-up.
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            error = False
            try:
                return func(*args, **kwargs)
            except Exception as e:
                display(Javascript(f'alert("❌ An error occurred: {str(e)}")'))
                error = True
                raise e
            finally:
                if show_popup and not error:
                    display(Javascript(f'alert("{message}")'))
        return wrapper
    return decorator

# Load Hugging Face token from token.txt if it exists
token_folder = glob.glob("token.txt")
if token_folder:
    with open(token_folder[0], "r") as f:
        token = f.read().strip()
    hf_token = token
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = token

@notify_when_done(show_popup=True, message="✅ Download complete!")
def download_model(list_of_models, output_dir="./"):
    for model_id in list_of_models:

        local_folder = os.path.join(output_dir, model_id)

        os.makedirs(local_folder, exist_ok=True)

        print(f"Downloading model files for {model_id} to {local_folder}...")

        # --- 2. Download the files ---
        snapshot_download(
            repo_id=model_id,
            local_dir=local_folder,
            ignore_patterns=["*.bin", "*.py", "*.md"],
        )

        print("\nDownload complete!")
        print(f"Model files are saved in: {os.path.abspath(local_folder)}")

## Download weights

In [ ]:
baseline_llm = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"
to_quantize_llm = "meta-llama/Llama-3.1-8B-Instruct"

download_model([baseline_llm, to_quantize_llm], output_dir="./weights")